In [1]:
%matplotlib inline

# importing libraries
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import cufflinks as cf
import seaborn as sns
import pandas as pd
import numpy as np
import quandl
import plotly
import time

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import Markdown, display
from matplotlib.ticker import FuncFormatter
from pandas.core.base import PandasObject
from datetime import datetime

# Setting pandas dataframe display options
pd.set_option("display.max_rows", 20)
pd.set_option('display.width', 800)
pd.set_option('max_colwidth', 800)

# Set plotly offline
init_notebook_mode(connected=True)

# Set matplotlib style
plt.style.use('seaborn')

# Set cufflinks offline
cf.go_offline()

# Defining today's Date
from datetime import date
today = date.today()

import PortfolioAnalyser as pl

In [2]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

In [126]:
def compute_time_series(dataframe):

#    INPUT: Dataframe of returns
#    OUTPUT: Growth time series starting in 100

    return (np.exp(np.log1p(dataframe).cumsum())) * 100

##################################################################################################################################################
def normalize(df):
    df = df.dropna()
    return (df / df.iloc[0]) * 100

##################################################################################################################################################
def compute_portfolio(quotes, weights, rebalance='yearly', start_amount=100):

    Start = str(quotes.index[0])[0:10]
    End = str(quotes.index[-1])[0:10]

    returns_all = pd.DataFrame()
    weights_all = pd.DataFrame()
    n = -1
    
    if rebalance == 'yearly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='A')
    elif rebalance == 'quarterly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='Q')
    elif rebalance == 'monthly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='M')
        
    if len(rebalance_dates) == 0:
            quotes_norm_periodo = normalize(quotes)
            quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
            quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
            returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
            returns_all = pd.concat([returns_all, returns_periodo])
            weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            
    else:          
        for i in np.arange(len(rebalance_dates)):
            n+=1
            break_point_future = rebalance_dates[n]
            break_point_past = rebalance_dates[n-1]
            # Até ao primeiro breakpoint
            if n == 0:
                quotes_norm_periodo = normalize(quotes[:break_point_future])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            # Break points a seguir
            if n > 0 and n < len(rebalance_dates):
                quotes_norm_periodo = normalize(quotes[break_point_past:break_point_future])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            # last break point
            if n == len(rebalance_dates) - 1:
                # print(n)
                quotes_norm_periodo = normalize(quotes[break_point_future:])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])

    portfolio = normalize(compute_time_series(returns_all) * (start_amount / 100))
    portfolio.drop_duplicates(inplace=True)
    weights_all = normalize(weights_all.div(weights_all.sum(axis=1), axis=0))
    portfolio.drop_duplicates(inplace=True)
    
    return portfolio

##################################################################################################################################################
def compute_portfolio_with_weights(quotes, weights, rebalance='yearly', start_amount=100):

    Start = str(quotes.index[0])[0:10]
    End = str(quotes.index[-1])[0:10]

    returns_all = pd.DataFrame()
    weights_all = pd.DataFrame()
    n = -1
    
    if rebalance == 'yearly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='A')
    elif rebalance == 'quarterly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='Q')
    elif rebalance == 'monthly':
        rebalance_dates = pd.bdate_range(start=Start, end=End, freq='M')
        
    if len(rebalance_dates) == 0:
            quotes_norm_periodo = normalize(quotes)
            quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
            quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
            returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
            returns_all = pd.concat([returns_all, returns_periodo])
            weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            
    else:          
        for i in np.arange(len(rebalance_dates)):
            n+=1
            break_point_future = rebalance_dates[n]
            break_point_past = rebalance_dates[n-1]
            # Até ao primeiro breakpoint
            if n == 0:
                quotes_norm_periodo = normalize(quotes[:break_point_future])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            # Break points a seguir
            if n > 0 and n < len(rebalance_dates):
                quotes_norm_periodo = normalize(quotes[break_point_past:break_point_future])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])
            # last break point
            if n == len(rebalance_dates) - 1:
                # print(n)
                quotes_norm_periodo = normalize(quotes[break_point_future:])
                quotes_pesos_periodo = quotes_norm_periodo * np.array(weights)
                quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
                returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
                returns_all = pd.concat([returns_all, returns_periodo])
                weights_all = pd.concat([weights_all, quotes_pesos_periodo])

    portfolio = normalize(compute_time_series(returns_all) * (start_amount / 100))
    portfolio.drop_duplicates(inplace=True)
    weights_all = normalize(weights_all.div(weights_all.sum(axis=1), axis=0))
    portfolio.drop_duplicates(inplace=True)
    
    return portfolio, weights

In [127]:
# download quotes
tickers = ['SPY', 'TLT']
Quotes = pd.DataFrame()
Start ='2006-03-31'
End = "2006-09-30"

Quotes = pd.read_csv('D:/GDrive/_GitHub/Backtester/Data/Cotacoes_diarias_all.csv', index_col='Date', parse_dates=True)[tickers].dropna()[Start:End]

Returns = Quotes.pct_change().dropna()

In [128]:
pl.compute_portfolio(quotes, [0.5, 0.5])

,Portfolio
2005-12-30,100.00
2005-12-31,100.00
2006-01-01,100.00
2006-01-02,100.00
2006-01-03,100.81
...,...
2007-12-27,115.94
2007-12-28,116.71
2007-12-29,116.71
2007-12-30,116.71


In [130]:
%%time
pd.set_option("display.max_rows", None)

Start = Quotes.index[0]
End = Quotes.index[-1]

quotes_norm = Quotes / Quotes.iloc[0]
returns = quotes_norm.pct_change()
portfolio_all = pd.DataFrame()

cashflows = 10
cashflows_dates = pd.bdate_range(start=Start, end=End, freq='M')
n = -1

for i in np.arange(len(cashflows_dates)):
    n+=1
    break_point_future = cashflows_dates[n]
    break_point_past = cashflows_dates[n - 1]
    # Até ao primeiro breakpoint
    if n == 0:
        portfolio = compute_portfolio(Quotes[: break_point_future], weights=[0.5, 0.5], start_amount=1000)
        portfolio.iloc[-1] += cashflows
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])
    # Break points a seguir
    if n > 0:
        portfolio = compute_portfolio(Quotes[break_point_past : break_point_future], weights=[0.5, 0.5], start_amount=valor_final_periodo)
        portfolio.iloc[-1] = cashflows + portfolio.iloc[-1]
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])
    # Último break point
    if n == len(cashflows_dates) - 1:
        portfolio = compute_portfolio(Quotes[break_point_future: ], weights=[0.5, 0.5], start_amount=valor_final_periodo)
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])

Wall time: 724 ms


In [131]:
pd.set_option("display.max_rows", 20)

def compute_portfolio_with_cashflows(data, weights, cashflows, start_amount=100):
    Start = Quotes.index[0]
    End = Quotes.index[-1]

    quotes_norm = Quotes / Quotes.iloc[0]
    returns = quotes_norm.pct_change()
    portfolio_all = pd.DataFrame()

    cashflows = cashflows
    cashflows_dates = pd.bdate_range(start=Start, end=End, freq='A')
    n = -1

    for i in np.arange(len(cashflows_dates)):
        n+=1
        break_point_future = cashflows_dates[n]
        break_point_past = cashflows_dates[n - 1]
        # Até ao primeiro breakpoint
        if n == 0:
            portfolio = compute_portfolio(Quotes[: break_point_future], weights=weights, start_amount=start_amount)
            portfolio.iloc[-1] += cashflows
            valor_final_periodo = portfolio.iloc[-1]
            portfolio_all = pd.concat([portfolio_all, portfolio])
        # Break points a seguir
        if n > 0:
            portfolio = compute_portfolio(Quotes[break_point_past : break_point_future], weights=weights, start_amount=valor_final_periodo)
            portfolio.iloc[-1] = cashflows + portfolio.iloc[-1]
            valor_final_periodo = portfolio.iloc[-1]
            portfolio_all = pd.concat([portfolio_all, portfolio])
        # Último break point
        if n == len(cashflows_dates) - 1:
            portfolio = compute_portfolio(Quotes[break_point_future: ], weights=weights, start_amount=valor_final_periodo)
            valor_final_periodo = portfolio.iloc[-1]
            portfolio_all = normalize(pd.concat([portfolio_all, portfolio]))
            
    return portfolio_all

In [141]:
# download quotes
tickers = ['IWDA', 'IE00B18GC888']
Quotes = pd.DataFrame()
Start ='2014-03-06'
End = "2020-12-31"

Quotes = pd.read_csv('D:/GDrive/_GitHub/Backtester/Data/Cotacoes_diarias_all.csv', index_col='Date', parse_dates=True)[tickers].dropna()[Start:End]

In [146]:
%%time
compute_portfolio(Quotes, weights=[0.5, 0.5])

Wall time: 38.9 ms


,0
2014-03-07,99.54
2014-03-10,99.47
2014-03-11,99.44
2014-03-12,99.05
2014-03-13,98.85
...,...
2020-10-09,153.44
2020-10-12,154.48
2020-10-13,154.89
2020-10-14,154.61


In [149]:
%%time
compute_portfolio_with_cashflows(Quotes, weights=[0.5, 0.5], cashflows=0)

Wall time: 120 ms


,0
2014-03-07,99.54
2014-03-10,99.47
2014-03-11,99.44
2014-03-12,99.05
2014-03-13,98.85
...,...
2020-10-09,153.44
2020-10-12,154.48
2020-10-13,154.89
2020-10-14,154.61


In [65]:
%%time
pd.set_option("display.max_rows", None)

Start = Quotes.index[0]
End = Quotes.index[-1]

quotes_norm = Quotes / Quotes.iloc[0]
returns = quotes_norm.pct_change()
portfolio_all = pd.DataFrame()

cashflows = 120
cashflows_dates = pd.bdate_range(start=Start, end=End, freq='A')
n = -1

for i in np.arange(len(cashflows_dates)):
    n+=1
    break_point_future = cashflows_dates[n]
    break_point_past = cashflows_dates[n - 1]
    # Até ao primeiro breakpoint
    if n == 0:
        portfolio = pl.compute_portfolio(Quotes[:break_point_future], weights=[0.5, 0.5], start_amount=1000)
        portfolio.iloc[-1] += cashflows
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])
    # Break points a seguir
    if n > 0 and n < len(cashflows_dates):
        portfolio = pl.compute_portfolio(Quotes[break_point_past:break_point_future], weights=[0.5, 0.5], start_amount=valor_final_periodo)
        portfolio.iloc[-1] = cashflows + portfolio.iloc[-1]
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])
    # Último break point
    if n == len(cashflows_dates) - 1:
        portfolio = pl.compute_portfolio(Quotes[break_point_future: ], weights=[0.5, 0.5], start_amount=valor_final_periodo)
        valor_final_periodo = portfolio.iloc[-1]
        portfolio_all = pd.concat([portfolio_all, portfolio])

Wall time: 159 ms


In [ ]:
Start = Quotes.index[0]
End = Quotes.index[-1]

quotes_norm = Quotes / Quotes.iloc[0]
returns = quotes_norm.pct_change()
returns_all = pd.DataFrame()
weights_all = pd.DataFrame()
cashflows_dates = pd.bdate_range(start=Start, end=End, freq='A')
n = -1

for i in np.arange(len(rebalance_dates)):
    n+=1
    break_point_future = rebalance_dates[n]
    break_point_past = rebalance_dates[n-1]
    # Até ao primeiro breakpoint
    if n == 0:
        quotes_norm_periodo = pl.normalize(Quotes[:break_point_future])
        quotes_pesos_periodo = quotes_norm_periodo * np.array([0.5, 0.5])
        quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
        returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
        returns_all = pd.concat([returns_all, returns_periodo])
        weights_all = pd.concat([weights_all, quotes_pesos_periodo])
    # Break points a seguir
    if n > 0:
        quotes_norm_periodo = pl.normalize(Quotes[break_point_past:break_point_future])
        quotes_pesos_periodo = quotes_norm_periodo * np.array([0.5, 0.5])
        quotes_pesos_periodo_soma = quotes_pesos_periodo.sum(axis=1)
        returns_periodo = quotes_pesos_periodo_soma.pct_change().dropna()
        returns_all = pd.concat([returns_all, returns_periodo])
        weights_all = pd.concat([weights_all, quotes_pesos_periodo])

compute_time_series(returns_all)

In [4]:
10471.07*0.001

10.47107